<a href="https://colab.research.google.com/github/ipmlab-ugr/Quantum-TimeSeries-Forecasting/blob/main/QNN_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quantum Machine Learning Regression with Qiskit
# This script demonstrates a quantum neural network regressor using Qiskit's machine learning components

# Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, EfficientSU2
from qiskit.primitives import Estimator
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_algorithms.optimizers import L_BFGS_B

# Constants
SEED = 42
N_QUBITS = 4  # Number of qubits for the quantum circuit
PCA_COMPONENTS = N_QUBITS  # PCA components should match number of qubits
REPS = 4  # Number of repetitions for feature map and ansatz

def set_random_seeds(seed):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)

def load_data():
    """Load training and test data using Google Colab's file upload"""
    from google.colab import files

    print("Please upload the file for X_train:")
    uploaded = files.upload()
    X_train = pd.read_csv(next(iter(uploaded.keys()))).to_numpy()

    print("Please upload the file for X_test:")
    uploaded = files.upload()
    X_test = pd.read_csv(next(iter(uploaded.keys()))).to_numpy()

    print("Please upload the file for y_train:")
    uploaded = files.upload()
    y_train = pd.read_csv(next(iter(uploaded.keys()))).to_numpy().flatten()

    print("Please upload the file for y_test:")
    uploaded = files.upload()
    y_test = pd.read_csv(next(iter(uploaded.keys()))).to_numpy().flatten()

    return X_train, X_test, y_train, y_test

def preprocess_data(X_train, X_test, y_train, y_test, n_components):
    """Preprocess data with normalization and PCA"""
    # Normalize features and targets
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    y_scaler = MinMaxScaler()
    y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1)).flatten()

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=n_components)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    return X_train, X_test, y_train_scaled, y_test_scaled, y_scaler

def build_quantum_circuit(n_qubits, reps):
    """Construct quantum circuit with feature map and ansatz"""
    feature_map = ZZFeatureMap(feature_dimension=n_qubits, reps=reps)
    ansatz = EfficientSU2(num_qubits=n_qubits, reps=reps)

    qc = QuantumCircuit(n_qubits)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    return qc, feature_map, ansatz

def train_qnn_regressor(X_train, y_train, circuit, feature_map, ansatz):
    """Train quantum neural network regressor"""
    estimator = Estimator()  # Use V2 Estimator

    qnn = EstimatorQNN(
        estimator=estimator,
        circuit=circuit,
        input_params=list(feature_map.parameters),
        weight_params=list(ansatz.parameters)
    )

    regressor = NeuralNetworkRegressor(
        neural_network=qnn,
        loss="squared_error",
        optimizer=L_BFGS_B(maxiter=10)
    )

    print("Training the Quantum Neural Network...")
    regressor.fit(X_train, y_train)

    return regressor

def evaluate_and_plot(regressor, X_test, y_test, y_scaler):
    """Make predictions and plot results"""
    print("Predicting with the Quantum Neural Network...")
    y_pred = regressor.predict(X_test)
    y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()

    plt.figure(figsize=(12, 6))
    plt.plot(y_test, label="Actual")
    plt.plot(y_pred, label="Predicted")
    plt.xlabel("Index")
    plt.ylabel("Target Value")
    plt.legend()
    plt.grid()
    plt.title("QNN: Actual vs Predicted")
    plt.show()

def main():
    # Set random seeds for reproducibility
    set_random_seeds(SEED)

    # Load data
    X_train, X_test, y_train, y_test = load_data()

    # Preprocess data
    X_train, X_test, y_train_scaled, y_test_scaled, y_scaler = preprocess_data(
        X_train, X_test, y_train, y_test, PCA_COMPONENTS
    )

    # Build quantum circuit
    qc, feature_map, ansatz = build_quantum_circuit(N_QUBITS, REPS)

    # Train QNN regressor
    regressor = train_qnn_regressor(X_train, y_train_scaled, qc, feature_map, ansatz)

    # Evaluate and plot results
    evaluate_and_plot(regressor, X_test, y_test, y_scaler)

if __name__ == "__main__":
    main()